In [45]:
import requests
import json
import unidecode


API_KEY = 'e4TJIDYJDObaWQC4Sg3hvTFuVcBXyONmv_CPrwd5F8afNcX1HzdEZFYm-433NPWzAJ6gBF_-0CRCRU-TN3t_5ne2r13babkdqSo_SAyBHHaMmHuuVjwyb0SPUWoSXXYx'
headers = {'Authorization': 'Bearer %s' % API_KEY}


'''
This file provides a suite of tools for searching the Yelp API to determine fraudulent activity in
restaurant receipts. Given user inputs for the restaurant name, location, and total bill, the program returns 
information about the restaurant and calculates the probability of the bill being the reported value.

Author: Michael Yan
'''
def main():
    
    restaurantName, location, moneySpent = getUserInput()
    
    params = {'term':restaurantName,'location':location}
    url='https://api.yelp.com/v3/businesses/search'
    name, price, rating, address, phone = searchAPI(params, url)
    
    
    print("\n*********************************************")
    print("Here is some information about the restuarant the employee went to:\n")
    
    print("Name:", name)
    print("Price range:", price)
    print("Rating:", rating)
    print("Address:", address)
    print("Phone:", phone)
    print("*********************************************\n")
    
    lower, upper = getDollarRange(price)
    print("At this price range, the employee should have spent between $", str(lower), " and $", str(upper))
    
    getProbability(moneySpent, lower, upper)
    

'''Get user input for the restaurant name, location, and total bill paid.'''    
def getUserInput():
    print("***Please answer the following questions.***")
    restaurantName, location, moneySpent = None, None, None
    while not restaurantName:
        restaurantName = input("What was the name of the restaurant? ")
    
    while not location:
        location = input("What was the location (general, e.g. Palo Alto)? ")
    
    while not moneySpent:
        moneySpent = input("How much was your total bill? ")
        
    return restaurantName, location, moneySpent


'''Query the Yelp API given the user input and return the information about the business'''
def searchAPI(params, url):
    # Making a get request to the API
    req=requests.get(url, params=params, headers=headers)
    parsed = json.loads(req.text)
    businesses = parsed["businesses"]
    
    business_list = [element for element in businesses]
    business = business_list[0]
    
    id = business["id"]
    url="https://api.yelp.com/v3/businesses/" + id + "/reviews"
    req = requests.get(url, headers=headers)
    parsed = json.loads(req.text)
    
    return unidecode.unidecode(business["name"]), business["price"], business["rating"], " ".join(business["location"]["display_address"]), business["phone"]


'''Returns tuple containing upper and lower bounds of price range'''
def getDollarRange(price):
    return (1,10) if (price == '$') else (11,30) if (price == '$$') else (31,60) if (price == '$$$') else (61, 'no limit')
    

def getProbability(moneySpent, lower, upper):
    moneySpent = float(moneySpent)
    lower = float(lower)
    upper = float(upper)
    print("The employee spent exactly $", str(moneySpent), " at this business. ")
    if(lower < moneySpent < upper):
        print("Statistically, the employee's expenditures are normal* at this price point.")     
    elif(moneySpent < lower):
        print("Statistically, the employee spent less money than expected at this business.")
    elif(moneySpent > upper):
       
        amtOver = (moneySpent - upper)/upper * 100
        
        
        print("Statistically, the employee's expenditures are abnormal at this price point.")
        print("The employee overspent the expected value by $", moneySpent-upper, " dollars")
        print("This is: ", amtOver,"% above the normal amount")
        
if __name__== "__main__":
  main()


***Please answer the following questions.***
What was the name of the restaurant? coupa cafe
What was the location (general, e.g. Palo Alto)? stanford
How much was your total bill? 20

*********************************************
Here is some information about the restuarant the employee went to:

Name: Coupa Cafe
Price range: $
Rating: 3.5
Address: 538 Ramona St Palo Alto, CA 94301
Phone: +16503226872
*********************************************

At this price range, the employee should have spent between $ 1  and $ 10
The employee spent exactly $ 20.0  at this business. 
Statistically, the employee's expenditures are abnormal at this price point.
The employee overspent the expected value by $ 10.0  dollars
This is:  100.0 % above the normal amount
